In [13]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import rospy
import time
from rt2_ass2.srv import Command,CommandRequest
from geometry_msgs.msg import Twist, Point
from nav_msgs.msg import Odometry
from tf import transformations
from rt2_ass2.srv import Position
import math
import actionlib
import actionlib.msg
import rt2_ass2.msg



robot_moving_random=False
robot_moving_command=False
rospy.init_node('notebook_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
pub_ = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
time.sleep(10)
rate = rospy.Rate(20)


linear_velocity = widgets.FloatSlider()
linear_velocity.min=0
linear_velocity.max=1
linear_velocity.description='Linear velocity of the direct control'
linear_velocity.value='0.5'

angular_velocity = widgets.FloatSlider()
angular_velocity.min=1
angular_velocity.max=2
angular_velocity.description='Angular velocity of the direct control'
angular_velocity.value='1.3'
display(linear_velocity,angular_velocity)
class Move:
    ind = 0
    def start(self, event):
        global robot_moving_random,robot_moving_command, linear_velocity, angular_velocity
        
        if not robot_moving_random:
            if robot_moving_command:
               self.stop(event)
            
            msg=CommandRequest()
            msg.command='start'
            msg.lin_vel=linear_velocity.value
            msg.ang_vel=angular_velocity.value
            ui_client(msg)
            robot_moving_random=True

    def stop(self, event):
        global robot_moving_random,robot_moving_command
        if robot_moving_random:
            
            msg=CommandRequest()
            msg.command='stop'
            ui_client(msg)
            robot_moving_random=False
            
        if robot_moving_command:
            twist_msg = Twist()
            pub_.publish(twist_msg)
            robot_moving_command=False

    def moveup(self, event):
        global robot_moving_random, robot_moving_command, linear_velocity
        
        if robot_moving_random:
            self.stop(event)
            time.sleep(1)
        
        twist_msg = Twist()
        twist_msg.linear.x = linear_velocity.value
        twist_msg.angular.z = 0
        pub_.publish(twist_msg)
        robot_moving_command=True
    def movedown(self, event):
        global robot_moving_random, robot_moving_command,linear_velocity 
        
        if robot_moving_random:
            self.stop(event)
            time.sleep(1)
        
        twist_msg = Twist()
        twist_msg.linear.x = -linear_velocity.value
        twist_msg.angular.z = 0
        pub_.publish(twist_msg)
        robot_moving_command=True
    def moveleft(self, event):
        global robot_moving_random, robot_moving_command,angular_velocity
        
        if robot_moving_random:
            self.stop(event)
            time.sleep(1)
        
        twist_msg = Twist()
        twist_msg.linear.x = 0
        twist_msg.angular.z = -angular_velocity.value
        pub_.publish(twist_msg)
        robot_moving_command=True
    def moveright(self, event):
        global robot_moving_random, robot_moving_command,angular_velocity

        if robot_moving_random:
            self.stop(event)
            time.sleep(1)
                    
        twist_msg = Twist()
        twist_msg.linear.x = 0
        twist_msg.angular.z = angular_velocity.value
        pub_.publish(twist_msg)
        robot_moving_command=True
        
        
robot = Move()
b1  = Button(description='Start random target',layout=Layout(width='auto', align="center", grid_area='b1'),style=ButtonStyle(button_color='lightblue'))
b2  = Button(description='Stop',layout=Layout(width='auto', grid_area='b2'),style=ButtonStyle(button_color='red'))
up  = Button(description='Move up',layout=Layout(width='auto', grid_area='up'),style=ButtonStyle(button_color='moccasin'))
down  = Button(description='Move down',layout=Layout(width='auto', grid_area='down'),style=ButtonStyle(button_color='moccasin'))
left  = Button(description='Rotate left',layout=Layout(width='auto', grid_area='left'),style=ButtonStyle(button_color='moccasin'))
right  = Button(description='Rotate right',layout=Layout(width='auto', grid_area='right'),style=ButtonStyle(button_color='moccasin'))

b1.on_click(robot.start)
b2.on_click(robot.stop)
up.on_click(robot.moveup)
left.on_click(robot.moveleft)
right.on_click(robot.moveright)
down.on_click(robot.movedown)

GridBox(children=[b1,  b2, up, left, right, down],layout=Layout(width='40%',grid_template_rows='auto auto auto',grid_template_columns='33% 33% 33%',grid_template_areas='''". up ."" left down right "" b1 b2 ."'''))


FloatSlider(value=0.5, description='Linear velocity of the direct control', max=1.0)

FloatSlider(value=1.3, description='Angular velocity of the direct control', max=2.0, min=1.0)

GridBox(children=(Button(description='Start random target', layout=Layout(grid_area='b1', width='auto'), style…